In [2]:
from functools import reduce
from types import SimpleNamespace as ns

import sympy
sympy.init_printing()

def block(f):
    return f()

In [3]:
def subs(e, **kwargs):
    _symbols = symbols(e)

    return e.subs({ _symbols[k]: v for k, v in kwargs.items() })

def symbols(e):
    return { s.name: s for s in e.free_symbols }


@block
def linscale():
    y, y_i, y_f, x, x_i, x_f = map(
        lambda name: sympy.Symbol(name, real=True),
        'y y_i y_f x x_i x_f'.split()
    )
    
    return sympy.Eq(y, y_i + ((y_f - y_i) / (x_f - x_i)) * x)

In [4]:
grid, row = ns(), ns()

grid.width = sympy.Symbol('grid.width', positive=True, real=True)
grid.node_count = sympy.Symbol('grid.node_count', positive=True, integer=True)
grid.row_count = sympy.Symbol('grid.row_count', positive=True, integer=True)
grid.margin_ratio = sympy.Symbol('grid.margin_ratio', positive=True, real=True)
grid.start_radius = sympy.Symbol('grid.start_radius', positive=True, real=True)
grid.end_radius = sympy.Symbol('grid.end_radius', positive=True, real=True)

row.index = sympy.Symbol('row.index', negative=False, integer=True)
row.extra_margin = sympy.Symbol('row.extra_margin', negative=False, real=True)
row.column_count = sympy.Symbol('row.column_count', positive=True, integer=True)
row.margin = sympy.Symbol('row.margin', negative=False, real=True)
row.radius = sympy.Symbol('row.radius', positive=True, real=True)
row.width = sympy.Symbol('row.width', positive=True, real=True)

# row.equation = (
#     row.extraMargin +
#     row.margin * (row.length + 1) +
#     2 * row.radius * row.length -
#     grid.width
# )

In [5]:
relations = ns()

relations.a = row.index < grid.row_count
relations.b = subs(linscale,
    y_i=grid.start_radius, y_f=grid.end_radius,
    x_i=0,                 x_f=grid.row_count - 1,

    y=row.radius,          x=row.index
)

relations.grid_width = sympy.Eq(grid.width,
    row.extra_margin +
    row.margin * (row.column_count + 1) +
    2 * row.radius * row.column_count
)

In [6]:
relations.row_index = row.index < grid.row_count

relations.radius = subs(linscale,
    y_i=grid.start_radius, y_f=grid.end_radius,
    x_i=0,                 x_f=grid.row_count - 1,

    y=row.radius,          x=row.index
)

relations.node_count = (
    grid.node_count <=

    sympy.Sum(
        sympy.solveset(
            relations.grid_width,
            row.column_count,
            domain=sympy.Reals
        )
        .subs({row.radius: relations.radius.rhs})
        .args[0],

        (row.index, 0, grid.row_count - 1)
    )
)

In [7]:
relations.node_count

                  grid.row_count - 1                                          
                        _____                                                 
                        ╲                                                     
                         ╲                                grid.width - row.ext
                          ╲          ─────────────────────────────────────────
                           ╲                               2⋅row.index⋅(grid.e
grid.node_count ≤          ╱         2⋅grid.start_radius + ───────────────────
                          ╱                                                gri
                         ╱                                                    
                        ╱                                                     
                        ‾‾‾‾‾                                                 
                    row.index = 0                                             

                                           
       

In [8]:
row_radius = sympy.Function('row_radius', positive=True)

eq = (
    grid.node_count <=
    sympy.Sum(row_radius(row.index), (row.index, 0, grid.row_count - 1))
)

eq

                  grid.row_count - 1                      
                         ___                              
                         ╲                                
                          ╲          row_radius(row.index)
grid.node_count ≤         ╱                               
                         ╱                                
                         ‾‾‾                              
                    row.index = 0                         

In [16]:
relations.grid_width

grid.width = 2⋅row.column_count⋅row.radius + row.extra_margin + row.margin⋅(ro
w.column_count + 1)

In [1]:
def a(n):
    return 1 + int(n / (5 ** 0.5))

In [4]:
[a(i) for i in range(20)]

[1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9]

```
0 -> 0
1 -> 1
2 -> -1
3 -> 2
4 -> -2
5 -> 3
6 -> -3
```

In [3]:
from math import ceil

def f(n):
    return ceil(n / 2) * (-1) ** (n + 1)

for i in range(0, 7):
    print(f'{i} -> {f(i)}')

0 -> 0
1 -> 1
2 -> -1
3 -> 2
4 -> -2
5 -> 3
6 -> -3


In [20]:
def plus_minus(n):
    return (-1) ** n

def zero_two(n):
    return 1 - plus_minus(n)

def slow_count(n):
    return n / 2 + zero_two(n) / 4

In [21]:
for n in range(20):
    print(f'{n} ->', slow_count(n) * plus_minus(n))

0 -> 0.0
1 -> -1.0
2 -> 1.0
3 -> -2.0
4 -> 2.0
5 -> -3.0
6 -> 3.0
7 -> -4.0
8 -> 4.0
9 -> -5.0
10 -> 5.0
11 -> -6.0
12 -> 6.0
13 -> -7.0
14 -> 7.0
15 -> -8.0
16 -> 8.0
17 -> -9.0
18 -> 9.0
19 -> -10.0


In [ ]:
0 -> 0 - 0
1 -> 1 - 1
2 -> 2 - 1
3 -> 3 - 2
4 -> 4 - 2
5 -> 5 - 3